In [1]:
# Import Dependencies
import pandas as pd
from sqlalchemy import create_engine
import datetime
import matplotlib.pyplot as plt
import numpy as np

import nltk
from nltk.corpus import stopwords
import re
import networkx
from textblob import TextBlob

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Create engine 
engine = create_engine("sqlite:///data/hashtag2020.db", echo=True)
conn = engine.connect()

2021-02-09 14:54:31,803 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-02-09 14:54:31,805 INFO sqlalchemy.engine.base.Engine ()
2021-02-09 14:54:31,807 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-02-09 14:54:31,807 INFO sqlalchemy.engine.base.Engine ()


In [3]:
#Create the table. If we use automap_base, the table must have a primary key
conn.execute("""
    CREATE TABLE IF NOT EXISTS tweet_counts (
        id integer PRIMARY KEY,
        created_at TEXT,
        tweet VARCHAR(280),
        likes integer,
        retweet_count integer,
        source VARCHAR(30),
        lat integer,
        long integer,
        city VARCHAR(100),
        state_code VARCHAR(5),
        hashtag VARCHAR(10),
        Date TEXT,
        Time TEXT,
        polarity integer,
        analysis VARCHAR(20)
        
    )
""")

2021-02-09 14:54:31,824 INFO sqlalchemy.engine.base.Engine 
    CREATE TABLE IF NOT EXISTS tweet_counts (
        id integer PRIMARY KEY,
        created_at TEXT,
        tweet VARCHAR(280),
        likes integer,
        retweet_count integer,
        source VARCHAR(30),
        lat integer,
        long integer,
        city VARCHAR(100),
        state_code VARCHAR(5),
        hashtag VARCHAR(10),
        Date TEXT,
        Time TEXT,
        polarity integer,
        analysis VARCHAR(20)
        
    )

2021-02-09 14:54:31,824 INFO sqlalchemy.engine.base.Engine ()
2021-02-09 14:54:31,829 INFO sqlalchemy.engine.base.Engine COMMIT


In [4]:
conn.execute("""
    CREATE TABLE IF NOT EXISTS sent_avg (
        avg_id integer PRIMARY KEY,
        city VARCHAR(100),
        lat integer,
        long integer,
        state_code VARCHAR(5),
        hashtag VARCHAR(10),
        Pol_Avg integer,
        analysis VARCHAR(20)
   
    )
""")

2021-02-09 14:54:31,839 INFO sqlalchemy.engine.base.Engine 
    CREATE TABLE IF NOT EXISTS sent_avg (
        avg_id integer PRIMARY KEY,
        city VARCHAR(100),
        lat integer,
        long integer,
        state_code VARCHAR(5),
        hashtag VARCHAR(10),
        Pol_Avg integer,
        analysis VARCHAR(20)
   
    )

2021-02-09 14:54:31,840 INFO sqlalchemy.engine.base.Engine ()
2021-02-09 14:54:31,846 INFO sqlalchemy.engine.base.Engine COMMIT


In [5]:
conn.execute("""
    CREATE TABLE IF NOT EXISTS sent_count (
        count_id integer PRIMARY KEY,
        hashtag VARCHAR(10),
        analysis VARCHAR(20),
        count integer
        
    )
""")

2021-02-09 14:54:31,855 INFO sqlalchemy.engine.base.Engine 
    CREATE TABLE IF NOT EXISTS sent_count (
        count_id integer PRIMARY KEY,
        hashtag VARCHAR(10),
        analysis VARCHAR(20),
        count integer
        
    )

2021-02-09 14:54:31,856 INFO sqlalchemy.engine.base.Engine ()
2021-02-09 14:54:31,860 INFO sqlalchemy.engine.base.Engine COMMIT


In [6]:
# Read in CSV's and build initial combined dataframe
trump_csv_path = "../hashtag_donaldtrump.csv"
biden_csv_path = "../hashtag_joebiden.csv"

In [7]:
trump_data_df = pd.read_csv(trump_csv_path, lineterminator='\n')
biden_data_df = pd.read_csv(biden_csv_path, lineterminator='\n')

In [8]:
trump_data_df["hashtag"]="Trump"
biden_data_df["hashtag"]="Biden"

In [9]:
tweet_df = pd.concat([trump_data_df, biden_data_df])

In [10]:
tweet_df["created_at"]=pd.to_datetime(tweet_df.created_at)

In [11]:
tweet_df.sort_values(by = 'created_at')

,created_at,tweet_id,tweet,likes,retweet_count,source,user_id,user_name,user_screen_name,user_description,...,user_location,lat,long,city,country,continent,state,state_code,collected_at,hashtag
0,2020-10-15 00:00:01,1.316529e+18,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0,TweetDeck,3.606665e+08,El Sol Latino News,elsollatinonews,🌐 Noticias de interés para latinos de la costa...,...,"Philadelphia, PA / Miami, FL",25.774270,-80.193660,NaN,United States of America,North America,Florida,FL,2020-10-21 00:00:00,Trump
0,2020-10-15 00:00:01,1.316529e+18,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0,TweetDeck,3.606665e+08,El Sol Latino News,elsollatinonews,🌐 Noticias de interés para latinos de la costa...,...,"Philadelphia, PA / Miami, FL",25.774270,-80.193660,NaN,United States of America,North America,Florida,FL,2020-10-21 00:00:00,Biden
1,2020-10-15 00:00:01,1.316529e+18,"Usa 2020, Trump contro Facebook e Twitter: cop...",26.0,9.0,Social Mediaset,3.316176e+08,Tgcom24,MediasetTgcom24,Profilo ufficiale di Tgcom24: tutte le notizie...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-21 00:00:00.373216530,Trump
2,2020-10-15 00:00:02,1.316529e+18,"#Trump: As a student I used to hear for years,...",2.0,1.0,Twitter Web App,8.436472e+06,snarke,snarke,"Will mock for food! Freelance writer, blogger,...",...,Portland,45.520247,-122.674195,Portland,United States of America,North America,Oregon,OR,2020-10-21 00:00:00.746433060,Trump
3,2020-10-15 00:00:02,1.316529e+18,2 hours since last tweet from #Trump! Maybe he...,0.0,0.0,Trumpytweeter,8.283556e+17,Trumpytweeter,trumpytweeter,"If he doesn't tweet for some time, should we b...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-21 00:00:01.119649591,Trump
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
970917,2020-11-08 23:59:54,1.325589e+18,"@sammelbis1998 @iheartmindy @bnorthg First, yo...",0.0,0.0,Twitter for iPad,7.687811e+08,Debra,drdeblk,MD. Autism specialist. Film lover. #DumpTrump2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-11-09 17:47:56.012674,Trump
970918,2020-11-08 23:59:55,1.325589e+18,OK just had to do it !\n#Trump #CatapultTrump ...,105.0,28.0,Twitter Web App,4.519791e+08,Dunken K Bliths,DunkenKBliths,"""Tesla owner's will be followed back""\n\n#Mach...",...,✔ Official Twitter Account,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-11-09 17:47:55.807500,Trump
922910,2020-11-08 23:59:56,1.325589e+18,@nbcbayarea Who doesn’t like dogs or any kind ...,1.0,1.0,Twitter for iPhone,1.182639e+08,Dianna Maria,DiannaMaria,"~ I am a very proud mom of a sweet, kind and c...",...,United States,39.783730,-100.445882,NaN,United States,North America,NaN,NaN,2020-11-09 17:46:06.939099,Trump
737089,2020-11-08 23:59:57,1.325589e+18,#JoeBiden 😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂 https://t.co/Ym...,0.0,0.0,Twitter for iPhone,1.148479e+08,Nataša,PewPeeew,My brain consists of coping mechanisms and vin...,...,Deutschland,51.083420,10.423447,NaN,Germany,Europe,NaN,NaN,2020-11-09 18:26:04.550843,Biden


In [12]:
usa = {"United States of America" : "United States"}
tweet_df['country'].replace(usa, inplace=True)

In [13]:
clean_df = tweet_df.dropna(how='any')

In [14]:
# Filter to data for United States only
clean_df = clean_df.loc[clean_df['country']== 'United States']

In [15]:
print("Count of duplicates: {}".format(clean_df.duplicated(subset=["tweet"]).sum()))

Count of duplicates: 21056


In [16]:
# Remove duplicate tweets, leaving only those with either hashtag not both
print("Original size of df: {}".format(len(clean_df)))
clean_df.drop_duplicates(subset=["tweet"], inplace=True, keep=False)
print("No duplicates size of df : {}".format(len(clean_df)))

Original size of df: 182384
No duplicates size of df : 141640


In [17]:
clean_df.reset_index(inplace=True)

In [18]:
clean_df['Date'] = clean_df['created_at'].dt.strftime('%m/%d/%Y')
clean_df['Time'] = clean_df['created_at'].dt.strftime('%H:%M:%S')

In [19]:
# Setting id column to use as primary key
clean_df = clean_df.rename(columns = {'index': 'id'}, inplace = False)

In [20]:
clean_df = clean_df[['id', 'created_at', 'tweet', 'likes', 'retweet_count', 'source', 'lat', 'long', 'city','state_code','hashtag', 'Date', 'Time']]

In [21]:
clean_df = clean_df[['created_at', 'tweet', 'likes', 'retweet_count', 'source', 'lat', 'long', 'city','state_code','hashtag', 'Date', 'Time']]

In [22]:
clean_df.reset_index(inplace=True)

In [23]:
clean_df = clean_df.rename(columns = {'index': 'id'}, inplace = False)
clean_df.head()

,id,created_at,tweet,likes,retweet_count,source,lat,long,city,state_code,hashtag,Date,Time
0,0,2020-10-15 00:00:02,"#Trump: As a student I used to hear for years,...",2.0,1.0,Twitter Web App,45.520247,-122.674195,Portland,OR,Trump,10/15/2020,00:00:02
1,1,2020-10-15 00:00:08,You get a tie! And you get a tie! #Trump ‘s ra...,4.0,3.0,Twitter for iPhone,38.894992,-77.036558,Washington,DC,Trump,10/15/2020,00:00:08
2,2,2020-10-15 00:00:26,#Trump #PresidentTrump #Trump2020LandslideVict...,3.0,5.0,Twitter for Android,32.717421,-117.162771,San Diego,CA,Trump,10/15/2020,00:00:26
3,3,2020-10-15 00:01:14,"#Trump: Nobody likes to tell you this, but som...",1.0,1.0,Twitter Web App,45.520247,-122.674195,Portland,OR,Trump,10/15/2020,00:01:14
4,4,2020-10-15 00:01:30,@karatblood @KazePlays_JC Grab @realDonaldTrum...,2.0,0.0,Twitter for iPhone,41.882823,-87.686307,Chicago,IL,Trump,10/15/2020,00:01:30


In [24]:
clean_df['id'].value_counts()

2047      1
34106     1
136552    1
44351     1
42302     1
         ..
125672    1
123625    1
129770    1
127723    1
0         1
Name: id, Length: 141640, dtype: int64

In [25]:
# Sentiment Analysis - code created by Try Suharso https://www.kaggle.com/trysuharso (2020) Pandas, NLTK, Re, TextBlob, Networkx. [Source code]. https://www.kaggle.com/trysuharso/us-election-2020-twitter-analysis

In [26]:
#cleaning the tweets
def clean_tweets(tweet):
 
    tweet = re.sub(r':', '', str(tweet))
    tweet = re.sub(r'‚Ä¶', '', str(tweet))
    tweet = re.sub('@[A-Za-z0–9]+', '', str(tweet))               
    tweet = re.sub('#', '', str(tweet)) # Removing '#' hash tag
    tweet = re.sub('https?:\/\/\S+', '', str(tweet)) # Removing hyperlink
    #replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ',str(tweet))
    tweet = str(tweet).lower()
    tweet = re.sub('\[.*?\]', '', tweet)
    tweet = re.sub('https?://\S+|www\.\S+', '', tweet)
    tweet = re.sub('<.*?>+', '', tweet)
    tweet = re.sub('\n', '', tweet)
    tweet = re.sub('\w*\d\w*', '', tweet)
    tweet = re.sub(r'#','',tweet)
    tweet = re.sub(r'RT[\s]+','',tweet)
    tweet = re.sub(r'[^\w]', ' ', tweet)
    tweet = re.sub('https', '', tweet)
    tweet = re.sub('https ', '', tweet)
    tweet = re.sub('co', '', tweet)

    return tweet

In [27]:
# Determine subjectivity, polarity, sentiment values of each tweet 
# ALLOW SEVERAL MINUTES TO COMPLETE
def getSubjectivity(tweet):
    try:
        return TextBlob(tweet).sentiment.subjectivity
    except:
        return None


# Create a function to get the polarity
def getPolarity(tweet):
    try:
        return  TextBlob(tweet).sentiment.polarity
    except:
        return None

def sentiment_calc(tweet):
    try:
        return TextBlob(tweet).sentiment
    except:
        return None

# Add sentiment analysis value columns to df
clean_df['edited_tweet'] = clean_df['tweet'].apply(clean_tweets)
clean_df['subjectivity'] = clean_df['edited_tweet'].apply(getSubjectivity)
clean_df['polarity'] = clean_df['edited_tweet'].apply(getPolarity)
clean_df['sentiment'] = clean_df['edited_tweet'].apply(sentiment_calc)

# Assign sentiment analysis category based on polarity value and add to df
def getAnalysis(score):
    if score < 0:
      return 'Negative'
    elif score == 0:
      return 'Neutral'
    else:
      return 'Positive'

clean_df['analysis'] = clean_df['polarity'].apply(getAnalysis)

In [28]:
clean_df.head()

,id,created_at,tweet,likes,retweet_count,source,lat,long,city,state_code,hashtag,Date,Time,edited_tweet,subjectivity,polarity,sentiment,analysis
0,0,2020-10-15 00:00:02,"#Trump: As a student I used to hear for years,...",2.0,1.0,Twitter Web App,45.520247,-122.674195,Portland,OR,Trump,10/15/2020,00:00:02,trump as a student i used to hear for years f...,0.333333,0.333333,"(0.3333333333333333, 0.3333333333333333)",Positive
1,1,2020-10-15 00:00:08,You get a tie! And you get a tie! #Trump ‘s ra...,4.0,3.0,Twitter for iPhone,38.894992,-77.036558,Washington,DC,Trump,10/15/2020,00:00:08,you get a tie and you get a tie trump s ral...,0.000000,0.000000,"(0.0, 0.0)",Neutral
2,2,2020-10-15 00:00:26,#Trump #PresidentTrump #Trump2020LandslideVict...,3.0,5.0,Twitter for Android,32.717421,-117.162771,San Diego,CA,Trump,10/15/2020,00:00:26,trump presidenttrump maga kag america ameri...,0.750000,0.500000,"(0.5, 0.75)",Positive
3,3,2020-10-15 00:01:14,"#Trump: Nobody likes to tell you this, but som...",1.0,1.0,Twitter Web App,45.520247,-122.674195,Portland,OR,Trump,10/15/2020,00:01:14,trump nobody likes to tell you this but some ...,0.595238,0.261905,"(0.2619047619047619, 0.5952380952380952)",Positive
4,4,2020-10-15 00:01:30,@karatblood @KazePlays_JC Grab @realDonaldTrum...,2.0,0.0,Twitter for iPhone,41.882823,-87.686307,Chicago,IL,Trump,10/15/2020,00:01:30,_jc grab by the balls amp chuck the bastar...,1.000000,0.000000,"(0.0, 1.0)",Neutral


In [29]:
# Creating dataframe for polarity average by city location for map visual
biden=clean_df[clean_df['hashtag']=='Biden']
trump=clean_df[clean_df['hashtag']=='Trump']

In [30]:
biden['Pol_Avg'] = biden.groupby('city').polarity.transform('mean')
trump['Pol_Avg'] = trump.groupby('city').polarity.transform('mean')


In [31]:
biden = biden[['city','lat', 'long', 'state_code', 'hashtag','Pol_Avg' ]]
trump = trump[['city','lat', 'long','state_code', 'hashtag','Pol_Avg' ]]

In [32]:
biden = biden.drop_duplicates(subset=['city'], keep='first')
trump = trump.drop_duplicates(subset=['city'], keep='first')

In [33]:
biden['analysis'] = biden['Pol_Avg'].apply(getAnalysis)
trump['analysis'] = trump['Pol_Avg'].apply(getAnalysis)

In [34]:
biden['analysis'].value_counts()

Positive    395
Negative     55
Neutral      42
Name: analysis, dtype: int64

In [35]:
trump['analysis'].value_counts()

Positive    325
Negative    117
Neutral      37
Name: analysis, dtype: int64

In [36]:
sent_avg = pd.concat([biden, trump])
sent_avg = sent_avg.reset_index(drop=True)

In [37]:
sent_avg.reset_index(inplace=True)
sent_avg = sent_avg.rename(columns = {'index': 'avg_id'}, inplace = False)

In [38]:
sent_avg.head()

,avg_id,city,lat,long,state_code,hashtag,Pol_Avg,analysis
0,0,Los Angeles,34.053691,-118.242766,CA,Biden,0.101237,Positive
1,1,New York,40.712728,-74.006015,NY,Biden,0.104129,Positive
2,2,Minneapolis,44.977300,-93.265469,MN,Biden,0.079609,Positive
3,3,Indianapolis,39.768333,-86.158350,IN,Biden,0.095884,Positive
4,4,Boca Raton,26.358688,-80.083098,FL,Biden,0.057974,Positive


In [39]:
# Creating dataframe for sentiment counts bar visual
biden_count=clean_df[clean_df['hashtag']=='Biden']
trump_count=clean_df[clean_df['hashtag']=='Trump']


In [40]:
biden_count = biden_count[['analysis', 'hashtag']]
trump_count = trump_count[['analysis', 'hashtag']]

In [41]:
biden_count["count"]=1
trump_count["count"]=1

In [42]:
biden_count = biden_count.groupby('analysis').count().reset_index()
trump_count = trump_count.groupby('analysis').count().reset_index()

In [43]:
biden_count['hashtag']="Biden"
trump_count['hashtag']="Trump"

In [44]:
sent_count = pd.concat([biden_count, trump_count])
sent_count = sent_count.reset_index(drop=True)

In [45]:
sent_count.reset_index(inplace=True)
sent_count = sent_count.rename(columns = {'index': 'count_id'}, inplace = False)

In [46]:
sent_count

,count_id,analysis,hashtag,count
0,0,Negative,Biden,9762
1,1,Neutral,Biden,29940
2,2,Positive,Biden,26064
3,3,Negative,Trump,17547
4,4,Neutral,Trump,31606
5,5,Positive,Trump,26721


In [47]:
clean_df = clean_df[['id', 'created_at', 'tweet', 'likes', 'retweet_count', 'source', 'lat', 'long', 'city','state_code','hashtag', 'Date', 'Time', 'polarity', 'analysis']]

In [48]:
clean_df.head()

,id,created_at,tweet,likes,retweet_count,source,lat,long,city,state_code,hashtag,Date,Time,polarity,analysis
0,0,2020-10-15 00:00:02,"#Trump: As a student I used to hear for years,...",2.0,1.0,Twitter Web App,45.520247,-122.674195,Portland,OR,Trump,10/15/2020,00:00:02,0.333333,Positive
1,1,2020-10-15 00:00:08,You get a tie! And you get a tie! #Trump ‘s ra...,4.0,3.0,Twitter for iPhone,38.894992,-77.036558,Washington,DC,Trump,10/15/2020,00:00:08,0.000000,Neutral
2,2,2020-10-15 00:00:26,#Trump #PresidentTrump #Trump2020LandslideVict...,3.0,5.0,Twitter for Android,32.717421,-117.162771,San Diego,CA,Trump,10/15/2020,00:00:26,0.500000,Positive
3,3,2020-10-15 00:01:14,"#Trump: Nobody likes to tell you this, but som...",1.0,1.0,Twitter Web App,45.520247,-122.674195,Portland,OR,Trump,10/15/2020,00:01:14,0.261905,Positive
4,4,2020-10-15 00:01:30,@karatblood @KazePlays_JC Grab @realDonaldTrum...,2.0,0.0,Twitter for iPhone,41.882823,-87.686307,Chicago,IL,Trump,10/15/2020,00:01:30,0.000000,Neutral


In [49]:
clean_df.to_sql("tweet_counts", conn, if_exists="append", index=False)


2021-02-09 14:57:56,275 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("tweet_counts")
2021-02-09 14:57:56,275 INFO sqlalchemy.engine.base.Engine ()
2021-02-09 14:57:56,364 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-02-09 14:57:57,971 INFO sqlalchemy.engine.base.Engine INSERT INTO tweet_counts (id, created_at, tweet, likes, retweet_count, source, lat, long, city, state_code, hashtag, "Date", "Time", polarity, analysis) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2021-02-09 14:57:57,972 INFO sqlalchemy.engine.base.Engine ((0, '2020-10-15 00:00:02.000000', "#Trump: As a student I used to hear for years, for ten years, I heard China! In 2019! And we have 1.5 and they don't know how many we have and I asked them how many do we have and they said 'sir we don't know.' But we have millions. Like 300 million.\n\nUm. What?", 2.0, 1.0, 'Twitter Web App', 45.5202471, -122.6741949, 'Portland', 'OR', 'Trump', '10/15/2020', '00:00:02', 0.3333333333333333, 'Positive'

In [50]:
sent_avg.to_sql("sent_avg", conn, if_exists="append", index=False)

2021-02-09 14:57:58,952 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("sent_avg")
2021-02-09 14:57:58,953 INFO sqlalchemy.engine.base.Engine ()
2021-02-09 14:57:58,955 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-02-09 14:57:58,962 INFO sqlalchemy.engine.base.Engine INSERT INTO sent_avg (avg_id, city, lat, long, state_code, hashtag, "Pol_Avg", analysis) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
2021-02-09 14:57:58,962 INFO sqlalchemy.engine.base.Engine ((0, 'Los Angeles', 34.0536909, -118.242766, 'CA', 'Biden', 0.10123705961934619, 'Positive'), (1, 'New York', 40.7127281, -74.0060152, 'NY', 'Biden', 0.10412915772957057, 'Positive'), (2, 'Minneapolis', 44.9772995, -93.2654692, 'MN', 'Biden', 0.07960914936599515, 'Positive'), (3, 'Indianapolis', 39.7683331, -86.1583502, 'IN', 'Biden', 0.09588398406129692, 'Positive'), (4, 'Boca Raton', 26.3586885, -80.08309840000003, 'FL', 'Biden', 0.057973575036075024, 'Positive'), (5, 'Seattle', 47.6038321, -122.3300624, 'WA', 'Biden', 0.

In [51]:
sent_count.to_sql("sent_count", conn, if_exists="append", index=False)

2021-02-09 14:57:58,982 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("sent_count")
2021-02-09 14:57:58,983 INFO sqlalchemy.engine.base.Engine ()
2021-02-09 14:57:58,984 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-02-09 14:57:58,985 INFO sqlalchemy.engine.base.Engine INSERT INTO sent_count (count_id, analysis, hashtag, count) VALUES (?, ?, ?, ?)
2021-02-09 14:57:58,985 INFO sqlalchemy.engine.base.Engine ((0, 'Negative', 'Biden', 9762), (1, 'Neutral', 'Biden', 29940), (2, 'Positive', 'Biden', 26064), (3, 'Negative', 'Trump', 17547), (4, 'Neutral', 'Trump', 31606), (5, 'Positive', 'Trump', 26721))
2021-02-09 14:57:58,986 INFO sqlalchemy.engine.base.Engine COMMIT
